# More cleaning (without the food.com data)

In [27]:
import pandas as pd
import numpy as np
import re
import pickle

In [10]:
nyt = pd.read_csv('data/nyt_cleaned.csv', sep = '|')

In [12]:
nyt.drop(columns = ['cleaned'], inplace = True)

In [13]:
allrecipes = pd.read_csv('data/allrecipes.csv', sep = '|')

In [14]:
allrecipes

,0,1,2
0,Betty&#39;s Famous Cinnamon Rolls,https://www.allrecipes.com/recipe/229402/betty...,['1 cup warm water (110 degrees F/45 degrees C...
1,Lemon Drop Shots,https://www.allrecipes.com/recipe/256748/lemon...,"['0.75 fluid ounce vodka (such as Absolut®)', ..."
2,Mississippi Pot Roast,https://www.allrecipes.com/recipe/260697/missi...,"['1 (3 pound) chuck roast', '0.5 (12 ounce) ja..."
3,Easy Chicken Tetrazzini for a Large Family,https://www.allrecipes.com/recipe/232165/easy-...,"['5 skinless, boneless chicken breasts', '2 ta..."
4,No-Yeast Pizza Crust,https://www.allrecipes.com/recipe/244638/no-ye...,"['1.3333333730698 cups all-purpose flour', '1 ..."
...,...,...,...
50611,Chocolate-Covered Coffee Beans,https://www.allrecipes.com/recipe/284194/choco...,"['4 ounces milk chocolate, chopped, divided', ..."
50612,Bracciole (Flank Steak Rolls),https://www.allrecipes.com/recipe/229505/bracc...,"['2 tablespoons olive oil', '0.5 onion, choppe..."
50613,Garlic-Smashed Potatoes,https://www.allrecipes.com/recipe/282326/garli...,"['5 pounds small red potatoes', '0.5 cup olive..."
50614,Chewy Cheesecake Cookies,https://www.allrecipes.com/recipe/284693/chewy...,"['0.5 cup unsalted butter, softened', '3 ounce..."


In [17]:
allrecipes = allrecipes.rename(columns = {'0': 'name', '1': 'url', '2': 'ingredients'}).copy()

In [18]:
allrecipes

,name,url,ingredients
0,Betty&#39;s Famous Cinnamon Rolls,https://www.allrecipes.com/recipe/229402/betty...,['1 cup warm water (110 degrees F/45 degrees C...
1,Lemon Drop Shots,https://www.allrecipes.com/recipe/256748/lemon...,"['0.75 fluid ounce vodka (such as Absolut®)', ..."
2,Mississippi Pot Roast,https://www.allrecipes.com/recipe/260697/missi...,"['1 (3 pound) chuck roast', '0.5 (12 ounce) ja..."
3,Easy Chicken Tetrazzini for a Large Family,https://www.allrecipes.com/recipe/232165/easy-...,"['5 skinless, boneless chicken breasts', '2 ta..."
4,No-Yeast Pizza Crust,https://www.allrecipes.com/recipe/244638/no-ye...,"['1.3333333730698 cups all-purpose flour', '1 ..."
...,...,...,...
50611,Chocolate-Covered Coffee Beans,https://www.allrecipes.com/recipe/284194/choco...,"['4 ounces milk chocolate, chopped, divided', ..."
50612,Bracciole (Flank Steak Rolls),https://www.allrecipes.com/recipe/229505/bracc...,"['2 tablespoons olive oil', '0.5 onion, choppe..."
50613,Garlic-Smashed Potatoes,https://www.allrecipes.com/recipe/282326/garli...,"['5 pounds small red potatoes', '0.5 cup olive..."
50614,Chewy Cheesecake Cookies,https://www.allrecipes.com/recipe/284693/chewy...,"['0.5 cup unsalted butter, softened', '3 ounce..."


In [19]:
len(nyt)

22075

In [21]:
df = pd.concat([allrecipes, nyt], axis = 0)

In [22]:
df.shape

(72691, 3)

In [23]:
ingredients = []
for row in df['ingredients']:
    row = row.replace('[\'', '\']')
    row = row.replace(']', '') 
    items = row.split(', ')
    for item in items: 
        ingredients.append(item.replace('\'', ''))

In [24]:
len(ingredients)

921997

In [25]:
len(set(ingredients))

180275

In [1]:
#ingredients[0:300]

Things to remove here:
- Everything between brackets
- All numbers
- Words that indicate sizes (cup/teaspoon/ounce/can/medium/pinch)
- Words that indicate processing (chopped, ground, sliced etc)

In [36]:
ingredients2 = []

Getting rid of brackets and numbers:

In [37]:
for item in ingredients: 
    nobrackets = re.sub('\([^\(]*\)','',item)
    nonumbers = re.sub('[0-9]*\.?[0-9]?', '', nobrackets)
    newitem = nonumbers.lower().strip()
    ingredients2.append(newitem)

Other words to ditch:

In [55]:
dumplist = [
    'cups',
    'cup',
    'teaspoons',    
    'teaspoon',
    'tablespoons',
    'tablespoon',
    'fluid ounces',
    'fluid ounce',
    ' ounces',
    ' ounce',
    ' cans',
    ' can',
    'packages',
    'package',
    'medium',
    ' large',
    ' small',
    ' jars',
    ' jar',
    ' pinches',
    ' pinch',
    ' chopped',
    'freshly',
    'ground',
    'sliced',
    'slices',
    'slice'
]

In [56]:
len(dumplist)

27

In [57]:
ingredients3 = []

In [58]:
for item in ingredients2:
    for entry in dumplist:
        item = item.replace(entry, '')
    ingredients3.append(item)

In [62]:
masterlist = list(set(ingredients3))

In [67]:
masterlist2 = []

In [73]:
for item in masterlist:
    for entry in dumplist:
        entry = entry.strip()
        item = re.sub(f'^{entry}s?','',item)
        item = item.replace('/','')
    masterlist2.append(item)

In [2]:
#masterlist2

In [76]:
with open("masterlist2.txt", 'w') as file:
    for line in masterlist2:
        file.write(f"{line}\n")

In [82]:
#ingredients[0:100]

In [77]:
masterlist3 = []
with open('masterlist2.txt', 'r') as file:
    lines = file.readlines()
    for line in lines: 
        masterlist3.append(line.replace('\n', ''))

In [78]:
len(masterlist3)

205329

In [79]:
len(set(masterlist3))

101261

In [80]:
masterlist3 = list(set(masterlist3))

In [81]:
with open("masterlist3.txt", 'w') as file:
    for line in masterlist3:
        file.write(f"{line}\n")

This is not really getting me anywhere - I still have to weed out 100,000 entries.

What I did instead was to weed out the first 1000 entries in this new list by hand. Once you've done 1000, you have a pretty good set of basic ingredients that you can use to clean up the others.

In [90]:
masterlist = []
with open("masterlist.txt", "r") as file: 
    lines = file.readlines()
    for line in lines: 
        masterlist.append(line.replace('\n', ''))

In [3]:
#masterlist

In [92]:
len(masterlist)

1180

In [93]:
len(set(masterlist))

990

In [94]:
masterlist = list(set(masterlist))

In [95]:
def simplify(row):

    rowresults = []
    row = row.replace('[\'', '\']')
    row = row.replace(']', '') 
    items = row.split(', ')
    for item in items: 
        for ingredient in masterlist:
            if ingredient in item: 
                rowresults.append(ingredient)
                break
    return rowresults

In [96]:
df['cleaned'] = df['ingredients'].apply(simplify)

In [97]:
df

,name,url,ingredients,cleaned
0,Betty&#39;s Famous Cinnamon Rolls,https://www.allrecipes.com/recipe/229402/betty...,['1 cup warm water (110 degrees F/45 degrees C...,"[water, active dry yeast, white sugar, milk, w..."
1,Lemon Drop Shots,https://www.allrecipes.com/recipe/256748/lemon...,"['0.75 fluid ounce vodka (such as Absolut®)', ...","[vodka, lemon juice, white sugar, lemon]"
2,Mississippi Pot Roast,https://www.allrecipes.com/recipe/260697/missi...,"['1 (3 pound) chuck roast', '0.5 (12 ounce) ja...","[pepper, pepper, salted butter, butter, black ..."
3,Easy Chicken Tetrazzini for a Large Family,https://www.allrecipes.com/recipe/232165/easy-...,"['5 skinless, boneless chicken breasts', '2 ta...","[chicken, butter, onion, mushroom, chicken, ce..."
4,No-Yeast Pizza Crust,https://www.allrecipes.com/recipe/244638/no-ye...,"['1.3333333730698 cups all-purpose flour', '1 ...","[all-purpose flour, baking powder, salt, milk,..."
...,...,...,...,...
22070,Mushrooms in Marsala Wine (Funghi Alla Marsala),https://cooking.nytimes.com/recipes/31-mushroo...,"['1 ounce dried mushrooms, preferably imported...","[mushroom, mushroom, water, oil, garlic, rosem..."
22071,Veal Scaloppine With Mushrooms Bordelaise,https://cooking.nytimes.com/recipes/30-veal-sc...,"['12 slices veal scaloppine, about 1 1/4 pound...","[veal, mushroom, oil, pea, corn, flour, pepper..."
22072,Mushroom and Meat Loaf,https://cooking.nytimes.com/recipes/28-mushroo...,"['1/2 pound mushrooms', '1 tablespoon butter',...","[mushroom, butter, onion, pork, veal, nutmeg, ..."
22073,Mushroom and Pepper Salad,https://cooking.nytimes.com/recipes/29-mushroo...,"['1 large sweet red pepper, about 1/2 pound', ...","[red pepper, green pepper, celery, mushroom, e..."


In [98]:
biglist = []
for i in range(len(df)):
    smallist = []
    for ingredient in masterlist: 
        if ingredient in df['cleaned'].iloc[i]:    
            smallist.append(1)
        else: 
            smallist.append(0)
    biglist.append(smallist)

In [99]:
matrix = pd.DataFrame(data = biglist, columns = masterlist)

In [101]:
matrix.to_csv('matrix2.csv', index = False)

In [102]:
df.reset_index(inplace = True, drop = True)

In [103]:
df

,name,url,ingredients,cleaned
0,Betty&#39;s Famous Cinnamon Rolls,https://www.allrecipes.com/recipe/229402/betty...,['1 cup warm water (110 degrees F/45 degrees C...,"[water, active dry yeast, white sugar, milk, w..."
1,Lemon Drop Shots,https://www.allrecipes.com/recipe/256748/lemon...,"['0.75 fluid ounce vodka (such as Absolut®)', ...","[vodka, lemon juice, white sugar, lemon]"
2,Mississippi Pot Roast,https://www.allrecipes.com/recipe/260697/missi...,"['1 (3 pound) chuck roast', '0.5 (12 ounce) ja...","[pepper, pepper, salted butter, butter, black ..."
3,Easy Chicken Tetrazzini for a Large Family,https://www.allrecipes.com/recipe/232165/easy-...,"['5 skinless, boneless chicken breasts', '2 ta...","[chicken, butter, onion, mushroom, chicken, ce..."
4,No-Yeast Pizza Crust,https://www.allrecipes.com/recipe/244638/no-ye...,"['1.3333333730698 cups all-purpose flour', '1 ...","[all-purpose flour, baking powder, salt, milk,..."
...,...,...,...,...
72686,Mushrooms in Marsala Wine (Funghi Alla Marsala),https://cooking.nytimes.com/recipes/31-mushroo...,"['1 ounce dried mushrooms, preferably imported...","[mushroom, mushroom, water, oil, garlic, rosem..."
72687,Veal Scaloppine With Mushrooms Bordelaise,https://cooking.nytimes.com/recipes/30-veal-sc...,"['12 slices veal scaloppine, about 1 1/4 pound...","[veal, mushroom, oil, pea, corn, flour, pepper..."
72688,Mushroom and Meat Loaf,https://cooking.nytimes.com/recipes/28-mushroo...,"['1/2 pound mushrooms', '1 tablespoon butter',...","[mushroom, butter, onion, pork, veal, nutmeg, ..."
72689,Mushroom and Pepper Salad,https://cooking.nytimes.com/recipes/29-mushroo...,"['1 large sweet red pepper, about 1/2 pound', ...","[red pepper, green pepper, celery, mushroom, e..."


In [104]:
df.to_csv('df.csv', sep = '|', index = False)

In [109]:
listy = []
for column in matrix.columns:
    if matrix[column].sum() == 0:
        listy.append(column)

In [111]:
len(listy)

391

In [108]:
max(listy)

24327